In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download("punkt")
import warnings
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\86183\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ag = pd.read_csv("../HomeWork1/ag.csv")
print(ag.shape)
ag.head()

(90000, 1)


,text
0,wall st. bears claw back into the black (reute...
1,carlyle looks toward commercial aerospace (reu...
2,oil and economy cloud stocks' outlook (reuters...
3,iraq halts oil exports from main southern pipe...
4,"oil prices soar to all-time record, posing new..."


In [3]:
sentences = [word_tokenize(text) for text in ag["text"].to_list()]
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, workers=4)
print(word2vec_model)
print(word2vec_model.wv["the"])

Word2Vec<vocab=25389, vector_size=100, alpha=0.025>
[-0.82592624 -0.1971812   0.05946393  0.71879333  0.00438883  1.4508421
  0.25752038 -1.2141894   0.4638295   1.0512335   0.3387962   1.0088668
 -0.3696457   0.7060422  -0.9622153  -0.09217576  1.410164    2.6029103
  0.08707961 -0.02023657  2.0257428   0.20089145  0.9982323   1.0454516
  1.6732069  -0.04954142 -2.288313    1.4232273   0.6731757   0.65620214
  1.3810301  -0.18968491  0.48619887 -0.61351854  0.6185155   0.34657052
  0.29178914  0.39643303 -0.5308937   1.4853861   2.30169     0.3284453
 -1.146065    0.5582236  -1.1595337   1.2471346  -0.44328767 -0.5926155
  0.12984386 -1.5251181  -0.3915281  -0.5550232  -0.9190749  -1.2711198
  1.12109     1.5189726  -1.0218965   0.70381373 -2.0246456  -0.17566851
 -0.6965267   0.08766004  0.22078095 -1.082654    2.3925478   1.1990571
  1.9888451  -0.5196234   1.8163198   0.513089   -1.4408895  -1.4309323
  0.37689212  0.8920007   1.3836681  -0.9727081   0.03014573  0.52733123
  0.2890

In [4]:
def get_text_vector(text, model=word2vec_model):
    words = word_tokenize(text.lower())
    word_vectors = []
    for word in words:
        if word.lower() in model.wv.key_to_index:
            word_vectors.append(model.wv[word.lower()])
        else:
            continue
    return np.mean(word_vectors, axis=0)

example_output = get_text_vector(text="The quick brown fox jumps over the lazy dog")
print(example_output)

[-1.4164463e-01  9.2357628e-02 -1.6924292e-01  1.3306247e-01
 -3.7872639e-02 -2.4995932e-02 -1.2057032e-01 -6.6590875e-01
  5.1347846e-01  4.4817686e-01  2.9777795e-01 -2.3529893e-01
 -1.3936751e-01  2.4590214e-01 -2.0378378e-01 -2.1049374e-01
  5.4308736e-01  6.3048255e-01 -4.7708783e-02 -4.7097963e-01
  6.6683108e-01  2.9616468e-02  5.2004218e-01  5.0716866e-03
  3.4972838e-01  7.1620941e-02 -1.7400810e-01  4.8853466e-01
  3.2987246e-01  1.5908208e-01  4.4249594e-01 -9.9320084e-02
 -1.7219466e-01 -1.8547949e-01 -1.7477772e-01  6.0103422e-01
 -1.1672221e-01  2.5408417e-01 -3.4748921e-01  7.3081225e-01
  6.4693487e-01  1.7703909e-01 -4.9722508e-01 -1.8030393e-01
  1.2769094e-01 -2.4747175e-01 -5.1036102e-01 -2.4882653e-01
 -7.0629008e-02 -4.0241587e-01 -4.0567580e-01 -7.2934315e-02
  7.9147078e-02 -2.9709288e-01  5.0851560e-01  5.4939747e-01
 -3.2788470e-01  5.4212970e-01 -6.8829721e-01  2.7876666e-01
 -1.3320294e-01 -1.5487905e-01 -1.4496875e-01 -2.2463322e-01
  2.7648333e-01  2.44405

In [5]:
df = pd.read_csv("../HomeWork1/nyt.csv")
print(df.shape)
df.head()

(11519, 2)


,text,label
0,(reuters) - carlos tevez sealed his move to ju...,sports
1,if professional pride and strong defiance can ...,sports
2,"palermo, sicily — roberta vinci beat top-seede...",sports
3,spain's big two soccer teams face a pair of it...,sports
4,the argentine soccer club san lorenzo complete...,sports


In [6]:
def split_df(df, train_size=0.8, val_size=0.1, test_size=0.1, random_state=42):
    train_df, temp_df = train_test_split(df, test_size=(1 - train_size), random_state=random_state)
    val_df, test_df = train_test_split(temp_df, test_size=test_size / (val_size + test_size), random_state=random_state)
    return train_df, val_df, test_df

In [7]:
train_df, val_df, test_df = split_df(df)
print(train_df.shape, val_df.shape, test_df.shape)

(9215, 2) (1152, 2) (1152, 2)


In [8]:
X_train = np.array([get_text_vector(text) for text in tqdm(train_df["text"].to_list(), total=len(train_df))])
X_val = np.array([get_text_vector(text) for text in tqdm(val_df["text"].to_list(), total=len(val_df))])
X_test = np.array([get_text_vector(text) for text in tqdm(test_df["text"].to_list(), total=len(test_df))])
print(X_train.shape, X_val.shape, X_test.shape)

100%|██████████| 1152/1152 [00:05<00:00, 206.67it/s]

(9215, 100) (1152, 100) (1152, 100)


In [9]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train, train_df["label"])

LogisticRegression(max_iter=2000)

In [10]:
test_preds = model.predict(X_test)
accuracy = accuracy_score(test_df["label"], test_preds)
macro_f1 = f1_score(test_df["label"], test_preds, average="macro")
micro_f1 = f1_score(test_df["label"], test_preds, average="micro")

print(f"Accuracy Score: {accuracy}")
print(f"Macro F1-Score: {macro_f1}")
print(f"Micro F1-Score: {micro_f1}")

Accuracy Score: 0.9696180555555556
Macro F1-Score: 0.9250232304082276
Micro F1-Score: 0.9696180555555556
